In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np
import scipy.signal as signal


# Delay Effect

In [2]:
def delay(x, delay_time, r, sr):

# A simple delay effect
# 
# - x: input signal
# - delay_time (in second): delay time
# - r: feedback gain
# - sr: sampling rate


    y = np.zeros(len(x))

    # length of delayline
    MAX_DELAY_LENGTH = sr*2

    # write pointer
    wp = 0

    # length of delayline
    delay_length = delay_time*sr;

    # fill delayline with random noise
    delayline = np.zeros(MAX_DELAY_LENGTH)
    
    # feed-back loop
    for n in range(len(x)):

        # read sample
        op = wp - delay_length
        if ( op < 0.00 ):
            op = op + MAX_DELAY_LENGTH

        rp = np.floor(op)
        rp_frac = op - rp
        rp = int(rp)
            
        # read sample using linear intepolation        
        if (rp >= (MAX_DELAY_LENGTH-1) | (rp == -1) ):
            tap_out = (1-rp_frac)*delayline[MAX_DELAY_LENGTH-1] + rp_frac*delayline[0]
        else:
            tap_out = (1-rp_frac)*delayline[rp-1] + rp_frac*delayline[rp]

        # write output
        delayline[wp] = r*tap_out + x[n]

        # update write pointer
        wp = wp + 1
        if wp >= MAX_DELAY_LENGTH:
            wp = 0

        y[n] = tap_out
        
    return y



In [40]:
x, sr = librosa.load("../audio/suzanne.wav") 
#print(sr)
#print(x.size)

delay_time = 1;
r = 0.9;
x = np.append(x, np.zeros(5*sr))
y_delay = delay(x, delay_time, r, sr)

ipd.Audio(y_delay, rate=sr)


# Chorus Effect

In [4]:
def chorus(x, delay_time, lfo_rate, lfo_depth, sr):

# A simple chorus effect
# 
# - x: input signal
# - delay_time (in second): delay time
# - lfo_depth (time): control the excursion of modulation
# - lfo_rate  (Hz): control the rate of modulation
# - sr: sampling rate


    y = np.zeros(len(x))

    # length of delayline
    MAX_DELAY_LENGTH = sr*2

    # write pointer
    wp = 0

    # length of delayline
    delay_length = delay_time*sr;

    # fill delayline with random noise
    delayline = np.zeros(MAX_DELAY_LENGTH);
    
    # initialize LFO settings
    lfo_depth_samples = lfo_depth*sr
    lfo_phase = 0
    lfo_phase_inc  = 2*np.pi*lfo_rate/sr;


    # feed-forward loop
    for n in range(len(x)):

        # LFO
        lfo_phase = lfo_phase + lfo_phase_inc
        if lfo_phase > 2*np.pi:
            lfo_phase = lfo_phase - 2*np.pi

        lfo_out = lfo_depth_samples*np.sin(lfo_phase)
        
        # read sample
        op = wp - delay_length
        op = op + lfo_out

        if ( op < 0.00 ):
            op = op + MAX_DELAY_LENGTH

        rp = np.floor(op)
        rp_frac = op - rp

        rp = int(rp)
        
        # read sample using linear intepolation        
        if (rp == (MAX_DELAY_LENGTH-1) | (rp == -1) ):
            tap_out = (1-rp_frac)*delayline[MAX_DELAY_LENGTH-1] + rp_frac*delayline[0]
        else:
            tap_out = (1-rp_frac)*delayline[rp-1] + rp_frac*delayline[rp]

        # write output
        delayline[wp] = x[n]

        # update write pointer
        wp = wp + 1
        if wp >= MAX_DELAY_LENGTH:
            wp = 0

#        print(tap_out, x[n])    
            
        y[n] = x[n] + tap_out
        
    return y



In [45]:
x, sr = librosa.load("../audio/AcousticGuitar.aif") 

#print(sr)
#print(x.size)
#print(x.size)

lfo_rate = 2 # Hz
lfo_depth = 0.001 #

delay_time = 0.02

x = np.append(x, np.zeros(1*sr))

y_chorus = chorus(x, delay_time, lfo_rate, lfo_depth, sr)

ipd.Audio(y_chorus, rate=sr)

In [46]:
def flanger(x, static_delay_time, variable_delay_time, lfo_rate, lfo_depth, sr):

# A simple flanger effect
# 
# - x: input signal
# - static_delay_time (in second): delay time
# - variable_delay_time (in second): delay time for the variable tap
# - lfo_depth (time): control the excursion of modulation
# - lfo_rate  (Hz): control the rate of modulation
# - sr: sampling rate

    y = np.zeros(len(x))

    # length of delayline
    MAX_DELAY_LENGTH = sr*2

    # write pointer
    wp = 0

    # length of delayline
    static_delay_length = static_delay_time*sr;
    variable_delay_length = variable_delay_time*sr;

    # fill delayline with random noise
    delayline = np.zeros(MAX_DELAY_LENGTH);
    
    # initialize LFO settings
    lfo_depth_samples = lfo_depth*sr
    lfo_phase = 0
    lfo_phase_inc  = 2*np.pi*lfo_rate/sr;


    # feed-forward loop
    for n in range(len(x)):

        # LFO
        lfo_phase = lfo_phase + lfo_phase_inc
        if lfo_phase > 2*np.pi:
            lfo_phase = lfo_phase - 2*np.pi

        lfo_out = lfo_depth_samples*np.sin(lfo_phase)
        
        # read sample
        op = wp - variable_delay_length
        op = op + lfo_out

        if ( op < 0.00 ):
            op = op + MAX_DELAY_LENGTH

        rp = np.floor(op)
        rp_frac = op - rp
        rp = int(rp)
        
        # variable tap out
        if (rp == (MAX_DELAY_LENGTH-1) | (rp == -1) ):
            variable_tap_out = (1-rp_frac)*delayline[MAX_DELAY_LENGTH-1] + rp_frac*delayline[0]
        else:
            variable_tap_out = (1-rp_frac)*delayline[rp-1] + rp_frac*delayline[rp]

            
        # static tap out
        op = wp - static_delay_length
        if ( op < 0.00 ):
            op = op + MAX_DELAY_LENGTH

        rp = np.floor(op)
        rp_frac = op - rp
        rp = int(rp)

        if (rp == (MAX_DELAY_LENGTH-1) | (rp == -1) ):
            static_tap_out = (1-rp_frac)*delayline[MAX_DELAY_LENGTH-1] + rp_frac*delayline[0]
        else:
            static_tap_out = (1-rp_frac)*delayline[rp] + rp_frac*delayline[rp+1]
    
        # write output
        delayline[wp] = x[n]

        # update write pointer
        wp = wp + 1
        if wp >= MAX_DELAY_LENGTH:
            wp = 0

        y[n] = 0.5*(static_tap_out + variable_tap_out)
        
    return y



In [52]:
x, sr = librosa.load("../audio/drumloop1.wav") 

#print(sr)
#print(x.size)
#print(x.size)

lfo_rate = 0.4# Hz
lfo_depth = 0.004 #

static_delay = 0.02
variable_delay = 0.0225

x = np.append(x, np.zeros(1*sr))

y_flanger = flanger(x, static_delay, variable_delay, lfo_rate, lfo_depth, sr)

ipd.Audio(y_flanger, rate=sr)

# Reverberation with Room impulse responses

In [33]:
x, sr = librosa.load("../audio/suzanne.wav") 
print(sr)
ir, sr2 = librosa.load("../audio/memchu_ir.wav") 
#ir, sr2 = librosa.load("../audio/lobby_ir.wav") 
#ir, sr2 = librosa.load("../audio/studio_ir.wav") 
print(sr2)

y = np.convolve(x, ir)

ipd.Audio(y, rate=sr)


22050
22050


# Physical Modeling Using the Karplus Strong Model

In [53]:
def karplus_strong(note, r, a, dur, sr):

# plucked string synthesis by the Karplus-Strong Algorithm
# 
# - note: note number
# - r: feedback gain, control the decay time
# - dur: duration of the note
# - sr: sampling rate    

    np.random.seed(1)

    f0 = 441*np.power(2,(note-69)/12)

    sampling_len = np.round(dur*sr)
    sampling_len = int(sampling_len)
    y = np.zeros(sampling_len)

    # length of delayline
    MAX_DELAY_LENGTH = np.ceil(sr/20)
    MAX_DELAY_LENGTH = int(MAX_DELAY_LENGTH)

    delay_length = sr/f0

    # write pointer
    wp = 0
    
    # fill delayline with random noise
    delayline = (np.random.rand(MAX_DELAY_LENGTH)-0.5)/2

    # delay element for lowpass filter
    tap_out_z = 0


    # feed-back loop
    for n in range(len(y)):

        # read sample
        op = wp - delay_length
        if ( op < 0.00 ):
            op = op + MAX_DELAY_LENGTH

        rp = np.floor(op)
        rp_frac = op - rp
        rp = int(rp)
            
        # read sample using linear intepolation        
        if (rp == (MAX_DELAY_LENGTH-1) | (rp == -1) ):
            tap_out = (1-rp_frac)*delayline[MAX_DELAY_LENGTH-1] + rp_frac*delayline[0]
        else:
            tap_out = (1-rp_frac)*delayline[rp-1] + rp_frac*delayline[rp]

        # lowpass filter
        lp_out = a*tap_out+(1-a)*tap_out_z;
        tap_out_z = tap_out;    
            
        # write output
        delayline[wp] = r*lp_out

        # update write pointer
        wp = wp + 1
        if wp >= MAX_DELAY_LENGTH:
            wp = 0

        y[n] = lp_out
        
    return y

In [58]:
sr = 44100

# from Yoon Sang's "Waiting for Happiness"
note = np.array([69, 76, 76, 69, 67,  74,  72,  65, 69, 72, 72, 71])
note_length = 0.28*np.array([1.0,  1.0,  1.0,  1.0,  1.5, 1.5, 2.0,  1.0,  1.0,  1.0,  1.5, 1.5])


x = np.zeros(1)
r = 0.995;
a = 0.7;

for i in range(len(note)):
    temp = karplus_strong(note[i], r, a, note_length[i], sr)
    x = np.append(x,temp)

ipd.Audio(x, rate=sr)


In [59]:
# add flanger

static_delay = 0.02
variable_delay = 0.025

lfo_rate = 1        # 0.01 Hz
lfo_depth = 0.002   # 0.001 or around

y1 = flanger(x, variable_delay, static_delay, lfo_rate, lfo_depth, sr)

ipd.Audio(y1, rate=sr)


In [60]:
# add delay

delay_time = 0.28
r = 0.6

y1 = np.append(y1, np.zeros(sr))
y2 = delay(y1, delay_time, r, sr)

ipd.Audio(y2, rate=sr)


